# Baseline Prediction 

https://towardsdatascience.com/multivariate-time-series-forecasting-using-random-forest-2372f3ecbad1

# <span style='background :khaki' > Install & Importa Libraries </span>

In [1]:
### ***Enviroment Preparation***
# Install Pandas
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install lightgbm

!pip install dtale

!pip install sktime
!pip install sklego

#!pip install skforecast

# Update pip -- WARNING Resolution
!python.exe -m pip install --upgrade pip

In [2]:
### ***Imports***
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import dtale

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from datetime import datetime
from datetime import timedelta

from statsmodels.tsa.stattools import adfuller, kpss

from sklego.preprocessing import RepeatingBasisFunction

from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import time
# Current Location.. !cd
#C:\Users\ghoyo\Desktop\TFM\Nuevo\Proyect

# Paths
data = r"C:\Users\ghoyo\Desktop\TFM4\Project\GeneratedDfs\dayly_rests_type.json"

#### <span style="background:skyblue"> Load Data <span>

In [3]:
df = pd.read_json(data)
df.shape

(11504, 2)

#### <span style="background:skyblue"> Little Analysis <span>


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11504 entries, 2017-04-04 14:06:56 to 2022-11-24 22:04:22
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   rest_type           11504 non-null  int64
 1   n_streams_listened  11504 non-null  int64
dtypes: int64(2)
memory usage: 269.6 KB


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
rest_type,11504.0,2.459840,1.685460,0.0,1.0,2.0,4.0,6.0
n_streams_listened,11504.0,7.129781,8.876177,1.0,2.0,5.0,9.0,191.0


In [6]:
df.head(4)

,rest_type,n_streams_listened
2017-04-04 14:06:56,1,2
2017-04-04 14:55:10,1,9
2017-04-04 16:28:27,2,7
2017-04-04 17:28:27,0,18


# <span style='background :khaki' > Baseline Method </span>

### <span style="background:lightgreen">  Baseline = Predict X Past Values <span>

Asuming the prediction of next day rest is going to be = 6 steps forward

https://www.theclickreader.com/multi-step-time-series-forecasting/

In [7]:
n_rests = 6
df.head(2)

,rest_type,n_streams_listened
2017-04-04 14:06:56,1,2
2017-04-04 14:55:10,1,9


#### <span style="background:violet"> Functions <span>

In [8]:
# persistence model
def model_persistence(x):
    return x

In [9]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits, numb):
        self.n_splits = n_splits
        self.numb = numb
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = stop-self.numb
            yield indices[start: mid], indices[mid + margin: stop]

### <span style="background:lightgreen"> Dummy Regresors - lag_1 <span>

In [10]:
# Create Base Lag

df_base = df.copy()
n_lags = 1

for i in range(n_lags):
    df_base['lag_'+ str(i+1)] = df_base['rest_type'].shift(i+1)

df_base.dropna(inplace=True)
df_base.tail(7)

,rest_type,n_streams_listened,lag_1
2022-11-22 07:18:52,5,2,3.0
2022-11-22 18:13:55,5,12,5.0
2022-11-23 06:53:53,5,13,5.0
2022-11-23 18:14:12,5,14,5.0
2022-11-23 21:31:19,3,5,5.0
2022-11-24 07:27:12,5,2,3.0
2022-11-24 22:04:22,5,11,5.0


In [11]:
# X DATA... 
cols = df_base.columns[df_base.columns.str.contains('lag')]
X = df_base[cols].squeeze()

# Y DATA SPLIT
y = df_base['rest_type']

# Other Variables
test_size = n_rests
predict = []

# Train, Predict and error.
for x in X[-test_size:]:
    yhat = model_persistence(x)
    predict.append(yhat)
    
acc = accuracy_score(y[-test_size:], predict)

In [12]:
print('Prediction = ', yhat, ' -- Vs -- ', 'Original = ', list(y[-test_size:])[0])
print('ACC = ', acc)

Prediction =  5.0  -- Vs --  Original =  5
ACC =  0.6666666666666666


#### <span style="background:skyblue"> Baseline CV </span>

#### <span style="background:skyblue"> TSCV </span>

In [13]:
# X DATA... 
cols = df_base.columns[df_base.columns.str.contains('lag')]
X = df_base[cols].squeeze()
# Y DATA SPLIT
y = df_base['rest_type']

# Other Variables
test_size = n_rests
predict = []
acc = []
#Split, Train, Predict and error.

tscv = TimeSeriesSplit(max_train_size=None, n_splits = 75, test_size=test_size)

for train_index, test_index in tscv.split(X):
    predict = []
    #print('TRAIN:', train_index, 'TEST:', test_index) 
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    for x in X_test:
        yhat = model_persistence(x)
        predict.append(yhat)
        
    #print('Prediction = ', predict, ' -- Vs -- ', 'Original = ', list(y_test[-test_size:]))
    acc.append(accuracy_score(y_test[-test_size:], predict))
    #print('ACC = ', acc[-1])
    
print('ACC Mean = ', np.mean(acc))
print('\n')


ACC Mean =  0.20888888888888887




#### <span style="background:skyblue"> BCV </span>

In [14]:
# X DATA... 
cols = df_base.columns[df_base.columns.str.contains('lag')]
X = df_base[cols].squeeze()
# Y DATA SPLIT
y = df_base['rest_type']

# Other Variables
test_size = n_rests
predict = []
acc = []
#Split, Train, Predict and error.

tscv = BlockingTimeSeriesSplit(n_splits = 75, numb = test_size)
for train_index, test_index in tscv.split(X):
    predict = []
    #print('TRAIN:', train_index, 'TEST:', test_index) 
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    for x in X_test:
        yhat = model_persistence(x)
        predict.append(yhat)
        
    #print('Prediction = ', predict, ' -- Vs -- ', 'Original = ', list(y_test[-test_size:]))
    acc.append(accuracy_score(y_test[-test_size:], predict))
    #print('ACC = ', acc[-1])
    
print('ACC Mean = ', np.mean(acc))
print('\n')


ACC Mean =  0.18222222222222223


